# MC_Issues framework demo

In [1]:
%pip install -q -e  ../. 
from directory_issues.issues.test_issues import IntIssue, StringIssue

from random import choice
from string import ascii_uppercase


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
lalal


## use metric_type.calculate_all(payload) get all issues found with a payload

In [2]:
refs = []
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s)
    refs.append(r)
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

LGXOOONWWNTKSD is too long
LGXOOONWWNTKSD contains duplicate letters
LGXOOONWWNTKSD does not contain the letter 'a'
##########
MOEUOVITJR contains duplicate letters
MOEUOVITJR does not contain the letter 'a'
##########
NFHUWIJXJYS is too long
NFHUWIJXJYS contains duplicate letters
NFHUWIJXJYS does not contain the letter 'a'
##########
LELLTQS is too short
LELLTQS contains duplicate letters
LELLTQS does not contain the letter 'a'
##########
XVOXMXRSSDCW is too long
XVOXMXRSSDCW contains duplicate letters
XVOXMXRSSDCW does not contain the letter 'a'
##########
EXGQJJYTYUI is too long
EXGQJJYTYUI contains duplicate letters
EXGQJJYTYUI does not contain the letter 'a'
##########
BPZRBXPHBMPRC is too long
BPZRBXPHBMPRC contains duplicate letters
BPZRBXPHBMPRC does not contain the letter 'a'
##########
NFXCZJPUJR contains duplicate letters
NFXCZJPUJR does not contain the letter 'a'
##########
FDSFBQLCG is too short
FDSFBQLCG contains duplicate letters
FDSFBQLCG does not contain the letter 'a'

## Use include_tags to only run on certain kinds of issue

In [3]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, include_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

ELBRDNFJPN contains duplicate letters
ELBRDNFJPN does not contain the letter 'a'
##########
VCXQD does not contain the letter 'a'
##########
NXRJUHKV does not contain the letter 'a'
##########
JFLQVZV contains duplicate letters
JFLQVZV does not contain the letter 'a'
##########
GXTMYLCMNXSM contains duplicate letters
GXTMYLCMNXSM does not contain the letter 'a'
##########
ZQNBCKZUOKJIK contains duplicate letters
ZQNBCKZUOKJIK does not contain the letter 'a'
##########
No issues for TCBLPAQM
##########
LUPFDOXOI contains duplicate letters
LUPFDOXOI does not contain the letter 'a'
##########
EVOFU does not contain the letter 'a'
##########
BUVMUKNTJBWICL contains duplicate letters
BUVMUKNTJBWICL does not contain the letter 'a'
##########


## And use exclude_tags to limit what kinds of issues you're checking for

In [4]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, exclude_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

VWGZP is too short
##########
XKYJPIZ is too short
##########
RNZPUTHTYQMSQ is too long
##########
DEPBGCCLV is too short
##########
SPFOWYA is too short
##########
RAPNFCBDG is too short
##########
PLTJONSTT is too short
##########
WQPUJMJWDLD is too long
##########
RLLUVVATZAKM is too long
##########
LEXGCZDW is too short
##########


## The framework tries to catch and report errors in issue calculation gracefully
Note that execution isn't halted- we just log an error, and report that the issue didn't calculate right. 

In [5]:
print(IntIssue.calculate_all(2, exclude_tags=["breaking"]))
print(IntIssue.calculate_all(3))
print(IntIssue.calculate_all(8, exclude_tags=["breaking"]))

[{'issue_name': 'is_even', 'template': '2 is even', 'tags': []}]
[{'issue_name': 'broken_issue', 'tags': ['breaking'], 'error': True, 'error_message': 'Intentional Error', 'template': "An error occurred while calculating 'broken_issue'"}]
[{'issue_name': 'is_even', 'template': '8 is even', 'tags': []}]


# Usage:

We shouldn't need very many extensions on the base issue class- we'll certainly need a SourceIssue, FeedIssue, and a CollectionIssue but that's probably it. 
Regardless, generating one is a two-liner: 

In [6]:
from directory_issues.issues import IssueBase
from typing import TypeVar, Tuple, Generic, Type, Dict, Callable, List, Any

class FloatIssue(IssueBase[float]):
    _ISSUES: Dict[str, Type["FloatIssue"]] = {}


Then actually implimenting the issue cases requires creating a class with just two methods- `calculate(payload)` and `render_template()`, then decorating it correctly:

In [7]:
@FloatIssue.register("decimal_places", tags=['decimal'])
class DecimalPlaces(FloatIssue):
    def calculate(self, payload:float) -> Tuple[bool, Dict]:        
        """
        NB the return signature: Tuple[bool, Any]
        bool is ultimately interpreted as "is_issue" - if false, the result is skipped
        Dict is the input provided to the render template
        """
        num, dec = str(payload).split(".")
        exp = len(dec)
    
        return exp > 2, {'value':payload, "exp":exp}

    def render_template(self):
        return f"{self.result['value']} has too many decimal places ({self.result['exp']})"


@FloatIssue.register("negative")
class NegativeFloat(FloatIssue):
    def calculate(self, payload:float):
        return payload < 0, {'value':payload}

    def render_template(self):
        return f"{self.result['value']} is negative"
            

In [8]:
for i in (1.2, -4.7 ,1.24525):
    r = FloatIssue.calculate_all(i)
    print(r)

[]
[{'issue_name': 'negative', 'template': '-4.7 is negative', 'tags': []}]
[{'issue_name': 'decimal_places', 'template': '1.24525 has too many decimal places (5)', 'tags': ['decimal']}]


At dev time, you should only be calling an issue from the calculate_all method- there's no need to ever call issue.calculate or issue.render_template directly. 
